In [20]:
sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("tensorflow_hub")
# sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1295, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



## Read CSV File and Get DataFrame (pyspark)

In [21]:
import pyspark.sql.functions as F
import re
from pyspark.sql.types import *

train_path = "s3://myprojectsentiment/data/train.csv"
test_path = "s3://myprojectsentiment/data/test.csv"

def readCSV(path):
    df = (spark.read
      .option("charset", "utf-8")
      .option("multiline", "true")
      .option("quote", '"')
      .option("header", "true")
      .option("escape", "\\")
      .option("escape", '"')
      .csv(path))
    df = df.select([F.col(col).alias(re.sub("[^0-9a-zA-Z$]+","",col)) for col in df.columns])
    df = df.withColumn('label', F.col('label').cast(DoubleType()))
    return df

train_df = readCSV(train_path)
test_df = readCSV(test_path)

train_df.printSchema()
train_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- content: string (nullable = true)
 |-- label: double (nullable = true)

+--------------------+-----+
|             content|label|
+--------------------+-----+
|Working with one ...|  0.0|
|Well...tremors I,...|  0.0|
|Ouch! This one wa...|  0.0|
|I've seen some cr...|  0.0|
|"Carriers" follow...|  0.0|
|I had been lookin...|  0.0|
|Effect(s) without...|  0.0|
|This picture star...|  0.0|
|I chose to see th...|  0.0|
|This film has to ...|  0.0|
|I felt brain dead...|  0.0|
|A young scientist...|  0.0|
|Inept, boring, an...|  0.0|
|From the first ti...|  0.0|
|I find it hard to...|  0.0|
|I actually saw Ch...|  0.0|
|I went to school ...|  0.0|
|I haven't seen th...|  0.0|
|I haven't seen an...|  0.0|
|One would think t...|  0.0|
+--------------------+-----+
only showing top 20 rows

In [22]:
train_data = train_df.toPandas().astype({'label': 'int32'})
test_data = test_df.toPandas().astype({'label': 'int32'})

train_examples, train_labels = train_data['content'],train_data['label']
test_examples, test_labels = test_data['content'],test_data['label']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
import numpy as np
train_examples = np.asarray(train_examples)
test_examples = np.asarray(test_examples)
train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
train_labels

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

## Neural Network

In [25]:
import tensorflow as tf
import tensorflow_hub as hub

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have two advantages:

we don't have to worry about text preprocessing,
we can benefit from transfer learning.
Here we will use a model from TensorFlow Hub called google/nnlm-en-dim50/2.

In [26]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.30287248,  0.03152315, -0.13984068,  0.25619656, -0.2741541 ,
         0.08599555, -0.26031104, -0.20547138, -0.17986763,  0.07881903,
         0.00308445, -0.14840184,  0.01973467,  0.27642438,  0.27174598,
        -0.31411722,  0.01907233,  0.16445579, -0.07911275, -0.35414273,
         0.15555151, -0.32292828,  0.11039259,  0.04350166,  0.0873228 ,
        -0.02004656, -0.51341045, -0.17435473,  0.15697853,  0.0375976 ,
        -0.04735049,  0.13688873,  0.17165697, -0.19878183, -0.11240551,
        -0.01976011,  0.327817  ,  0.06841931,  0.03370661, -0.40007097,
        -0.01591244, -0.1395479 , -0.2554904 ,  0.31316546, -0.07917289,
        -0.4301121 , -0.02118236, -0.08704337, -0.10098184, -0.07780349],
       [ 0.92870265,  0.10013989, -0.12779008,  0.7637115 ,  0.18019122,
        -0.11815798,  0.4077077 , -0.11286174, -0.5793166 ,  0.70438725,
        -0.11078805,  0.42786193, -0.04827447, -0.07137473, -0.35188085,
 

In [27]:
train_labels[:10]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Build the model

In [28]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________

In [29]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Split validation data

In [30]:
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Epoch 1/40
30/30 [==============================] - 15s 451ms/step - loss: 0.5214 - accuracy: 0.8238 - val_loss: 1.7405 - val_accuracy: 2.0000e-04
Epoch 2/40
30/30 [==============================] - 13s 422ms/step - loss: 0.4358 - accuracy: 0.8314 - val_loss: 1.5255 - val_accuracy: 3.0000e-04
Epoch 3/40
30/30 [==============================] - 13s 428ms/step - loss: 0.3888 - accuracy: 0.8278 - val_loss: 1.3652 - val_accuracy: 0.0350
Epoch 4/40
30/30 [==============================] - 13s 436ms/step - loss: 0.3139 - accuracy: 0.8603 - val_loss: 1.1026 - val_accuracy: 0.2770
Epoch 5/40
30/30 [==============================] - 12s 402ms/step - loss: 0.2424 - accuracy: 0.9049 - val_loss: 0.9428 - val_accuracy: 0.4647
Epoch 6/40
30/30 [==============================] - 13s 442ms/step - loss: 0.1774 - accuracy: 0.9414 - val_loss: 0.9343 - val_accuracy: 0.5209
Epoch 7/40
30/30 [==============================] - 13s 422ms/step - loss: 0.1292 - accuracy: 0.9625 - val_loss: 0.8577 - val_accuracy

In [ ]:
results = model.evaluate(test_examples, test_labels)

print(results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

782/782 [==============================] - 5s 7ms/step - loss: 1.0363 - accuracy: 0.7834
[1.0363460779190063, 0.7833999991416931]